<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Single-Record Ingestion for Multiclass Classification (Classification and AUC Metrics)

In this tutorial, we'll outline how to send single-record predictions (scores + labels) and actuals from multiclass models to Arize in order to calculate classification metrics and AUC for our model. Multiclass classification models are defined as a classification model with more than two classes. Each example can only be labeled as one class. For more information on multiclass ingestion, please see our documentation <a href="https://docs.arize.com/arize/model-types/multiclass-classification">here</a>. For a full list of all model types, please see our documentation <a href="https://docs.arize.com/arize/">here</a>.

## Install and Import Dependencies

In [ ]:
!pip install -q arize
from arize.api import Client
from arize.utils.types import ModelTypes, Environments

import concurrent.futures as cf
import pandas as pd
import json
import urllib.request

## Download and Display Data
For this tutorial, we will use a sample JSON file representing a single prediction. 

In [ ]:
file_url = "https://storage.googleapis.com/arize-assets/documentation-sample-data/data-ingestion/multiclass-classification-assets/multiclass-single-example.json"

with urllib.request.urlopen(file_url) as f:
    record = json.load(f)

print(json.dumps(record, indent=3))

## Create Arize Client
Sign up/login to your Arize account <a href="https://app.arize.com/auth/login">here</a>. Find your <a href="https://docs.arize.com/arize/api-reference/arize.log/client">Space and API keys</a>. Copy/paste into the cell below. 

In [ ]:
SPACE_KEY = "SPACE_KEY"  # update value here with your Space Key
API_KEY = "API_KEY"  # update value here with your API key

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print(
        "✅ Import and Setup Arize Client Done! Now we can start using Arize!"
    )

## Log Data to Arize
In order to send the probability/propensity for each class label in the prediction (i.e., the prediction scores), we need to fan out the single inference into a prediction for each class value. We'll use <a href="https://docs.arize.com/arize/sending-data/model-schema-reference#9.-tags">tags</a> to identify which class value is associated with each prediction score, which we can then filter on in the Arize platform. The prediction label will remain the same across all predictions and will represent what the model actually predicted for that specific record. The actual label will also remain the same across all predictions and will be the record's true actual label. The example below shows how 1 record will be fanned out into three predictions - one for each class value. 

#### Example prediction

**Inference**
```
record = {
  "prediction_id": "pred_123",
  "prediction_scores": {
    "first_class": 0.75,
    "business_class": 0.15,
    "economy_class": 0.10
  },
  "predicted_class": "first_class",
  "actual": "first_class"
}
```

**Predictions Sent to Arize**

| prediction_id | prediction_label | prediction_score | tag | actual_label |
| --- | ----------- | ------| ----- | --------- | 
| pred_123_first_class | first_class | 0.75 | first_class | first_class | 
| pred_123_business_class | first_class | 0.15 | business_class | first_class | 
| pred_123_economy_class | first_class | 0.10 | economy_class | first_class | 


***
#### Code to Implement the Above
Log the record using the <a href="https://docs.arize.com/arize/sending-data-to-arize/data-ingestion-methods/sdk-reference/python-sdk/arize.log">single-record API</a>.

In [ ]:
resps = []

id_prefix = record["prediction_id"]
pred_label = record["class_prediction"]
actual_label = record["actual_class"]
feature_dict = record["features"]

for label in record["prediction_scores"]:
    # get ID for label + prediction combo
    id_ = f"{id_prefix}_{label}"

    # log example to Arize
    future = arize_client.log(
        model_id="multiclass-classification-and-auc-metrics-single-record-ingestion-tutorial",
        model_version="1.0",
        model_type=ModelTypes.SCORE_CATEGORICAL,
        environment=Environments.PRODUCTION,
        prediction_id=id_,
        prediction_label=(pred_label, record["prediction_scores"][label]),
        actual_label=actual_label,
        features=feature_dict,
        tags={"label": label}
    )
    resps.append(future)

for future in cf.as_completed(resps):
    res = future.result(timeout=10)
    if res.status_code == 200:
        print(f"✅ future completed with response code {res.status_code}")
    else:
        print(
            f"❌ future failed with response code {res.status_code}, {res.text}"
        )